In [3]:
# 📦 Imports
import pandas as pd
import requests
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox, Output, Tab
from IPython.display import display, clear_output

pd.set_option("display.max_columns", None)

# 📥 Load data from the government API
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3&limit=5000'
response = requests.get(url)
data = response.json()
data_df = pd.DataFrame(data['result']['records'])

# Convert year to numeric for plotting
data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

# 🎛️ Filter widgets
manufacturer_dropdown = widgets.Dropdown(
    options=[''] + sorted(data_df['tozeret_nm'].dropna().unique().tolist()),
    description='Manufacturer:',
    style={'description_width': 'initial'}
)

model_dropdown = widgets.Dropdown(
    options=[''],
    description='Model:',
    style={'description_width': 'initial'}
)

trim_dropdown = widgets.Dropdown(
    options=[''],
    description='Trim:',
    style={'description_width': 'initial'}
)

def update_model_options(change):
    selected = change['new']
    if selected:
        model_dropdown.options = [''] + sorted(data_df[data_df['tozeret_nm'] == selected]['kinuy_mishari'].dropna().unique())
    else:
        model_dropdown.options = ['']
    trim_dropdown.options = ['']

def update_trim_options(change):
    selected_manufacturer = manufacturer_dropdown.value
    selected_model = change['new']
    if selected_manufacturer and selected_model:
        trims = data_df[
            (data_df['tozeret_nm'] == selected_manufacturer) &
            (data_df['kinuy_mishari'] == selected_model)
        ]['ramat_gimur'].dropna().unique()
        trim_dropdown.options = [''] + sorted(trims)
    else:
        trim_dropdown.options = ['']

manufacturer_dropdown.observe(update_model_options, names='value')
model_dropdown.observe(update_trim_options, names='value')

# 🧮 Tab 1 – Statistics
tab1_out = Output()
with tab1_out:
    display(data_df.describe(include='all'))

# 📋 Tab 2 – Data Table
tab2_out = Output()
with tab2_out:
    display(data_df.head(50))

# 📈 Tab 3 – Yearly Bar Chart
tab3_out = Output()
with tab3_out:
    year_counts = data_df['shnat_yitzur'].value_counts().sort_index()
    fig, ax = plt.subplots(figsize=(10, 5))
    year_counts.plot(kind='bar', ax=ax)
    ax.set_title("Number of Vehicles per Year")
    ax.set_xlabel("Year")
    ax.set_ylabel("Count")
    plt.tight_layout()
    plt.show()

# 🔎 Tab 4 – Filtered View
filter_out = Output()

def update_filtered_output(change=None):
    filter_out.clear_output()
    t = manufacturer_dropdown.value
    k = model_dropdown.value
    r = trim_dropdown.value

    filtered_df = data_df.copy()
    if t:
        filtered_df = filtered_df[filtered_df['tozeret_nm'] == t]
    if k:
        filtered_df = filtered_df[filtered_df['kinuy_mishari'] == k]
    if r:
        filtered_df = filtered_df[filtered_df['ramat_gimur'] == r]

    with filter_out:
        print(f"Total Records: {len(filtered_df)}")
        display(filtered_df.head(10))

manufacturer_dropdown.observe(update_filtered_output, names='value')
model_dropdown.observe(update_filtered_output, names='value')
trim_dropdown.observe(update_filtered_output, names='value')

tab4_out = VBox([manufacturer_dropdown, model_dropdown, trim_dropdown, filter_out])

# 🧷 Combine Tabs
tabs = Tab(children=[tab1_out, tab2_out, tab3_out, tab4_out])
tab_titles = ['📊 Statistics', '📋 Table', '📈 Yearly Chart', '🔎 Filters']
for i in range(len(tab_titles)):
    tabs.set_title(i, tab_titles[i])

display(tabs)
